In [38]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import sklearn.model_selection
from numpy.random import random
from sklearn.preprocessing import StandardScaler, MinMaxScaler

c0=sns.color_palette()[0]
c1=sns.color_palette()[1]
c2=sns.color_palette()[2]

cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

def points_plot(ax, Xtr, Xte, ytr, yte, clf, mesh=True, colorscale=cmap_light, 
                cdiscrete=cmap_bold, alpha=0.1, psize=10, zfunc=False, predicted=False):
    h = .02
    X=np.concatenate((Xtr, Xte))
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))

    #plt.figure(figsize=(10,6))
    if zfunc:
        p0 = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 0]
        p1 = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
        Z=zfunc(p0, p1)
    else:
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    ZZ = Z.reshape(xx.shape)
    if mesh:
        plt.pcolormesh(xx, yy, ZZ, cmap=cmap_light, alpha=alpha, axes=ax)
    if predicted:
        showtr = clf.predict(Xtr)
        showte = clf.predict(Xte)
    else:
        showtr = ytr
        showte = yte
    ax.scatter(Xtr[:, 0], Xtr[:, 1], c=showtr-1, cmap=cmap_bold, 
               s=psize, alpha=alpha,edgecolor="k")
    # and testing points
    ax.scatter(Xte[:, 0], Xte[:, 1], c=showte-1, cmap=cmap_bold, 
               alpha=alpha, marker="s", s=psize+10)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    return ax,xx,yy

def points_plot_prob(ax, Xtr, Xte, ytr, yte, clf, colorscale=cmap_light, 
                     cdiscrete=cmap_bold, ccolor=cm, psize=10, alpha=0.1):
    ax,xx,yy = points_plot(ax, Xtr, Xte, ytr, yte, clf, mesh=False, 
                           colorscale=colorscale, cdiscrete=cdiscrete, 
                           psize=psize, alpha=alpha, predicted=True) 
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=ccolor, alpha=.2, axes=ax)
    cs2 = plt.contour(xx, yy, Z, cmap=ccolor, alpha=.6, axes=ax)
    #plt.clabel(cs2, fmt = '%2.1f', colors = 'k', fontsize=14, axes=ax)
    return ax 

In [21]:
df = pd.read_csv('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/processed/EDA.csv')


In [22]:
df.head().T

,0,1,2,3,4
season,2020,2020,2020,2020,2020
team,100 Thieves,Cloud9,Counter Logic Gaming,Dignitas,Evil Geniuses
kills_pg,10.898,15.6491,9.72973,9.28571,11.1667
deaths_pg,12.7551,9.26316,14.3784,11.0476,12.95
firstblood_pg,0.44898,0.561404,0.405405,0.47619,0.483333
assists_pg,24.7551,36.3684,24.4054,22.1667,26.1833
wardskilled_pg,49.0204,41.7719,46.5946,49.0952,47.0167
controlwardsbought_pg,42.9796,37.7895,44.1892,43.1667,47.3333
firsttower_pg,0.510204,0.754386,0.324324,0.404762,0.433333
towers_pg,5.61224,8,4.05405,4.61905,6.08333


In [23]:
df = df.set_index(['season','team'])

In [24]:
df.head()

kills_pg  deaths_pg  firstblood_pg  assists_pg  wardskilled_pg  controlwardsbought_pg  firsttower_pg  towers_pg  firsttothreetowers_pg  totalgold_pg  earnedgold_pg  goldspent_pg  minionkills_pg  monsterkills_pg  inhibitors_pg  Win_Percentage  Wins  Losses  kills  assists  firstblood  firstdragon  mountains  oceans  gamesplayed  deaths  wardskilled  controlwardsbought  firsttower  towers  firsttothreetowers  totalgold  earnedgold  goldspent  minionkills  \
season team                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
2020   100 Thieves           10.897959  12.755102       0.448980   24.755102       49.020408              42.979592       0.510204   5.612245               0.367347  57166.857143   35227.510204  53744.040816      854.571429       205.959184       1.020408        0.408163  20.0    29.0  534.0   1213.0        22.0         24.0       30.0    39.0         49.0   625.0       2402.0              2106.0        25.0   275.0                18.0  2801176.0   1726148.0  2633458.0      41874.0   
       Cloud9                15.649123   9.263158       0.561404   36.368421       41.771930              37.789474       0.754386   8.000000               0.754386  58425.631579   37973.438596  52644.842105      796.052632       221.631579       1.508772        0.771930  44.0    13.0  892.0   2073.0        32.0         38.0       42.0    38.0         57.0   528.0       2381.0              2154.0        43.0   456.0                43.0  3330261.0   2164486.0  3000756.0      45375.0   
       Counter Logic Gaming   9.729730  14.378378       0.405405   24.405405       46.594595              44.189189       0.324324   4.054054               0.270270  57389.864865   34882.405405  54264.459459      896.594595       199.594595       0.540541        0.216216   8.0    29.0  360.0    903.0        15.0         13.0       14.0    16.0         37.0   532.0       1724.0              1635.0        12.0   150.0                10.0  2123425.0   1290649.0  2007785.0      33174.0   
       Dignitas               9.285714  11.047619       0.476190   22.166667       49.095238              43.166667       0.404762   4.619048               0.333333  57318.428571   34914.142857  54120.047619      886.809524       217.928571       0.571429        0.357143  15.0    27.0  390.0    931.0        20.0         19.0       14.0    27.0         42.0   464.0       2062.0              1813.0        17.0   194.0                14.0  2407374.0   1466394.0  2273042.0      37246.0   
       Evil Geniuses         11.166667  12.950000       0.483333   26.183333       47.016667              47.333333       0.433333   6.083333               0.516667  57701.816667   35752.533333  53983.800000      854.000000       208.866667       1.050000        0.483333  29.0    31.0  670.0   1571.0        29.0         34.0       33.0    36.0         60.0   777.0       2821.0              2840.0        26.0   365.0                31.0  3462109.0   2145152.0  3239028.0      51240.0   

                             monsterkills  inhibitors  
season team                                            
2020   100 Thieves                10092.0        50.0  
       Cloud9                     12633.0        86.0  
       Counter Logic Gaming        7385.0        20.0  
       Dignitas                    9153.0        24.0  
       Evil Geniuses              12532.0        63.0

In [25]:
df.shape

(47, 37)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 47 entries, (2020, '100 Thieves') to (2014, 'Team SoloMid')
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   kills_pg               47 non-null     float64
 1   deaths_pg              47 non-null     float64
 2   firstblood_pg          47 non-null     float64
 3   assists_pg             47 non-null     float64
 4   wardskilled_pg         47 non-null     float64
 5   controlwardsbought_pg  47 non-null     float64
 6   firsttower_pg          47 non-null     float64
 7   towers_pg              47 non-null     float64
 8   firsttothreetowers_pg  47 non-null     float64
 9   totalgold_pg           47 non-null     float64
 10  earnedgold_pg          47 non-null     float64
 11  goldspent_pg           47 non-null     float64
 12  minionkills_pg         47 non-null     float64
 13  monsterkills_pg        47 non-null     float64
 14  inhibitors_pg       

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the data into a training and test set.
Xlr, Xtestlr, ylr, ytestlr = train_test_split(df.drop(columns='Win_Percentage').values, 
                                              (df.Win_Percentage > 0.5).values,random_state=5)

clf = LogisticRegression()
# Fit the model on the trainng data.
clf.fit(Xlr, ylr)
# Print the accuracy from the testing data.
print(accuracy_score(clf.predict(Xtestlr), ytestlr))

0.8333333333333334


/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [39]:
scaler = StandardScaler()
scaler.fit(Xlr)
X_tr_scaled = scaler.transform(Xlr)
X_te_scaled = scaler.transform(Xtestlr)

In [41]:
clf.fit(X_tr_scaled, ylr)
# Print the accuracy from the testing data.
print(accuracy_score(clf.predict(X_te_scaled), ytestlr))

1.0


In [28]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

def cv_score(clf, x, y, score_func=accuracy_score):
    result = 0
    nfold = 5
    for train, test in KFold(nfold).split(x): # split data into train/test groups, 5 times
        clf.fit(x[train], y[train]) # fit
        result += score_func(clf.predict(x[test]), y[test]) # evaluate score function on held-out data
    return result / nfold # average

In [29]:
clf = LogisticRegression()
score = cv_score(clf, Xlr, ylr)
print(score)

0.8571428571428571


/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

In [30]:
#the grid of parameters to search over
Cs = [0.001, 0.1, 1, 10, 100]
max_score = 0

# your turn
for C in Cs:
        clf = LogisticRegression(C=C)
        score = cv_score(clf, Xlr, ylr)
        if score > max_score:
            max_score = score
            best_C = C
        #print(C)
        print(max_score)
        print(best_C)

/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

0.8571428571428571
0.001


/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

0.8571428571428571
0.001
0.8571428571428571
0.001


/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

0.8571428571428571
0.001
0.8571428571428571
0.001


/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

In [31]:
clf1 = LogisticRegression(C=10)
clf1.fit(Xlr, ylr)
ypred = clf1.predict(Xtestlr)
accuracy_score(ypred, ytestlr)

/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8333333333333334

In [32]:
# your turn
from sklearn.model_selection import GridSearchCV

param_grid = {"C": [0.0001, 0.001, 0.1, 1, 10, 100]}
clf2 = LogisticRegression()

logreg_cv = GridSearchCV(clf2, param_grid, cv=5, scoring = 'accuracy')

# Fit it to the data
logreg_cv.fit(Xlr,ylr)

# Print the tuned parameters and score
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
print("Best score is {}".format(logreg_cv.best_score_))

logreg_cv.best_estimator_, logreg_cv.best_params_, logreg_cv.best_score_

/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

Tuned Logistic Regression Parameters: {'C': 0.0001}
Best score is 0.8571428571428571


/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

(LogisticRegression(C=0.0001, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 {'C': 0.0001},
 0.8571428571428571)

In [33]:
ypred = logreg_cv.predict(Xtestlr)
accuracy_score(ypred, ytestlr)

0.75

In [34]:
from sklearn.model_selection import train_test_split

def do_classify(clf, parameters, indf, featurenames, targetname, target1val, standardize=False, train_size=0.8):
    subdf=indf[featurenames]
    if standardize:
        subdfstd=(subdf - subdf.mean())/subdf.std()
    else:
        subdfstd=subdf
    X=subdfstd.values
    y=(indf[targetname].values==target1val)*1
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=train_size)
    clf = cv_optimize(clf, parameters, Xtrain, ytrain)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    print("Accuracy on training data: {:0.2f}".format(training_accuracy))
    print("Accuracy on test data:     {:0.2f}".format(test_accuracy))
    return clf, Xtrain, ytrain, Xtest, ytest

In [16]:
def cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=5):
    gs = sklearn.model_selection.GridSearchCV(clf, param_grid=parameters, cv=n_folds)
    gs.fit(Xtrain, ytrain)
    print("BEST PARAMS", gs.best_params_)
    best = gs.best_estimator_
    return best

In [35]:
clf_l, Xtrain_l, ytrain_l, Xtest_l, ytest_l  = do_classify(LogisticRegression(), 
                                                           {"C": [0.01, 0.1, 1, 10, 100]}, 
                                                           df, ['kills_pg', 'towers_pg'], 'Win_Percentage',0.5)

BEST PARAMS {'C': 0.01}
Accuracy on training data: 0.97
Accuracy on test data:     1.00


/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

  FitFailedWarning)
/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

  FitFailedWarning)
/Users/dankang/opt/anaconda3/lib

In [36]:
clf_l.predict_proba(Xtest_l)

array([[0.97344208, 0.02655792],
       [0.97309833, 0.02690167],
       [0.97250422, 0.02749578],
       [0.97310718, 0.02689282],
       [0.97290517, 0.02709483],
       [0.97262542, 0.02737458],
       [0.97333584, 0.02666416],
       [0.972689  , 0.027311  ],
       [0.9726222 , 0.0273778 ],
       [0.97252055, 0.02747945]])